## Ratings attempt

In [1]:
from sklearn.datasets import fetch_20newsgroups
twenty_train = fetch_20newsgroups(subset='train',shuffle=True, random_state=42, categories =['talk.politics.misc'])

In [2]:
import pandas as pd
texts = pd.DataFrame(twenty_train.data)[0]

In [3]:
from gabriel.archangel import *

In [ ]:
archangel = Archangel(api_key = '')

In [13]:
# archangel = Archangel(api_key = api_key)
task_description = "I am analyzing political lean in online news forums."
attribute_dict = {
    'talks about politics': "The 'talks about politics' attribute quantifies the extent to which the text is focused on political issues, debates, or events. It is high when politics is a central theme of the discussion, medium when politics is mentioned alongside other topics, and low or zero when politics is not mentioned at all.",
    'democratic leaning': "The 'democratic leaning' attribute assesses the degree to which the text aligns with or supports values, policies, or figures associated with the Democratic Party. It is high when there is clear support or positive sentiment towards Democratic positions, low when there is minimal or indirect support, and zero when there is no alignment with Democratic views. You should infer this and rate if there in any implicit or explicit support for Democratic views. ",
    'republican leaning': "The 'republican leaning' attribute evaluates the extent to which the text aligns with or supports values, policies, or figures associated with the Republican Party. It is high when there is evident support or positive sentiment towards Republican positions, low when the support is minimal or indirect, and zero when there is no alignment with Republican views. You should infer this and rate if there in any implicit or explicit support for Republican views.",
    'exhibits anger': "The 'exhibits anger' attribute measures the presence and intensity of anger in the text. It is high when the text displays strong feelings of anger or frustration, especially towards political figures, policies, or events; medium when anger is present but not the dominant emotion; and low or zero when there is no anger.",
    'exhibits rationality': "The 'exhibits rationality' attribute determines the level of logical reasoning, evidence-based arguments, and objective analysis in the text. It is high when the text is characterized by clear, rational thought processes and a focus on facts; medium when rationality is present but mixed with subjective opinions; and low or zero when the text lacks rational argumentation or is primarily emotional."
}
output_df = await archangel.run_analysis(texts = texts.sample(10),
                        attribute_dict = attribute_dict,
                        mode = 'ratings',  # 'ratings' or 'classification'
                        num_runs = 1,
                        save_folder = '/Users/elliottmokski/Downloads/',
                        file_name = 'analysis_results.csv',
                        model = 'o3-mini',
                        reasoning_model=True,
                        reset_files = True,
                        task_description = task_description)

Estimated cost for this run: $0.1527 (includes reasoning, scaling factor=1.0)
Created new file at /Users/elliottmokski/Downloads/analysis_results.csv
Model used: o3-mini
Total prompts to process: 10
Truncated 0 prompts out of 10 (0.00%).


Processing prompts: 100%|██████████| 10/10 [00:24<00:00,  2.41s/it]

Results saved to /Users/elliottmokski/Downloads/analysis_results_responses.csv
All responses saved to /Users/elliottmokski/Downloads/analysis_results_responses.csv

Time taken summary (in seconds):
0th percentile: 11.20
10th percentile: 12.82
25th percentile: 15.25
50th percentile: 16.86
75th percentile: 19.33
90th percentile: 23.56
95th percentile: 23.57
99th percentile: 23.58
100th percentile: 23.59


In [6]:
res = pd.read_csv('/Users/elliottmokski/Downloads/analysis_results.csv')

## Classification attempt

In [7]:
import pandas as pd
techs = pd.read_csv('/Users/elliottmokski/Documents/Dynamo Revision/filtered_at_least_one_tech.csv')
techs_sample = techs.sample(25)

In [8]:
from archangel import *

In [9]:
attribute_dict = {
    "general class of technology": """'general class of technology' means the entity is a broad type of technology, of which there are many specific versions. For example, the entities 'electric power', 'steam engine', 'PC', 'airplane', 'artillery', 'smartphone', 'antibiotics' etc would all be classified as 'general class of technology' because they are all technology terms which encompass many more specific technologies that themselves would be well-known.""",
    
    "specific technology": """'specific technology' means the entity is a specific type of tech, but not an explicit tech product. For example, 'safety pin', 'object-oriented programming', 'computer mouse', 'diesel engine', 'floating-gate MOSFET', 'sunglasses', etc all belong to the 'specific technology' class. Critically, any specific name of a tech commercial product should be classified as 'specific tech product', not 'specific technology', which should only be generic versions of a specific tech that are not company specific.""",
    
    "specific tech product": """'specific tech product' means the entity is an explicit tech product that was built by a company or organization. For example, 'iPhone', 'M1 Garand', 'Band-Aid', 'Python', 'Ford Model T', 'Google', 'Prozac', etc would all be classified as 'specific tech product'. Importantly, 'search engine', 'integrated circuit', and 'SSRIs' would instead belong in the 'specific technology' class, because they are not products of a specific company. A 'specific tech product' must be company specific version of a technology. If the entity is not uniquely created by one company or organization, it should be classified as 'specific technology'.""",
    
    "tech process": """'tech process' means the entity is an invented process on how to make or use a technology. For example, 'Bessemer process', 'assembly line', 'interchangeable parts', various mining and agricultural practices, and so on would be classified as 'tech process' because they were not specific technologies but technological methodologies of manufacturing, service, or agriculture which were invented. These should be fairly specific processes.""",
    
    "fabrication": """'fabrication' means the entity is something that was created or synthesized by scientists or inventors, and doesn't occur in nature commonly. Examples are fabricated materials, synthesized drugs,""",
    
    "purely scientific discovery": """'purely scientific discovery' means the entity is a purely scientific discovery, like 'gravity', 'electromagnetism', 'cell theory', 'information theory', 'redox reactions', 'Young's double slit experiment', etc. Importantly, this category does NOT include discoveries that are also technologies or fabrications, like 'electromagnet', 'electrification', 'laser', various materials, etc.""",
    
    "inventor": """'inventor' means the entity is the name of a tech inventor, like 'Thomas Edison' or 'Steve Jobs'.""",
    
    "company": """'company' means the entity is the name of a company, such as 'Bell Labs', 'Apple', 'Standard Oil', etc. If the entity name can be considered a specific tech product as well as a company name, like 'Google', it should be put in the 'specific tech product' class, not here.""",
    
    "infrastructure": """'infrastructure' means the entity is the name of a specific building, train, road, or other specific infrastructure project. Examples would include 'Berlin U-Bahn', 'Beijing Subway', 'Los Angeles International Airport', 'I-95', 'Aswan Dam', 'Chrysler Building', etc. Crucially, this class must not be applied to broader infrastructure technologies, such as 'subway transportation', 'hydroelectric power', 'high speed rail', 'reinforced concrete', etc; these all belong in their respective tech classes.""",
    
    "certainly not a technology": """'certainly not a technology' means the entity is obviously not a technology and not related to technology at all. You should use this for entities which are not at all in the realm of technologies. Examples include 'frog', 'Kylie Jenner', 'standup comedy', and other entities which are very patently not any of the above categories and are not plausibly technologies."""
}

In [14]:
archangel = Archangel(api_key = api_key)
classification_output_df = await archangel.run_analysis(texts = techs_sample['processed_title'],
                        attribute_dict = attribute_dict,
                        mode = 'classification',  # 'ratings' or 'classification'
                        num_runs = 2,
                        save_folder = '/Users/elliottmokski/Downloads/',
                        file_name = 'classification_analysis_results.csv',
                        model = 'o3-mini',
                        n_parallels = 100,
                        reset_files = True)

Creating a new file at /Users/elliottmokski/Downloads/classification_analysis_results.csv
Estimated cost: 0.263 USD
Model used: o3-mini
Total prompts to process: 25
Truncated 0 prompts out of 25 (0.00%).


Processing prompts: 100%|██████████| 25/25 [00:21<00:00,  1.18it/s]

Results saved to /Users/elliottmokski/Downloads/classification_analysis_results_responses.csv
All responses saved to /Users/elliottmokski/Downloads/classification_analysis_results_responses.csv

Time taken summary (in seconds):
0th percentile: 1.92
10th percentile: 3.23
25th percentile: 3.99
50th percentile: 7.72
75th percentile: 15.25
90th percentile: 16.83
95th percentile: 18.91
99th percentile: 20.14
100th percentile: 20.38


# ELO Attempt

In [3]:
import asyncio
from gabriel.elo_ratings_code import *
from gabriel.openai_api_calls import OpenAIClient  # Your existing OpenAIClient

# 1) Create the OpenAI client
client = OpenAIClient(api_key=)

# 2) Define texts
texts = [
    "Some text about climate science and findings.",
    "A news snippet about a politician's stance on climate change.",
    "An academic paper abstract discussing greenhouse gas effects.",
    "A blog post with personal opinions and anecdotes on weather patterns."
]

# 3) Provide axis and user clarifications
axis_of_interest = "climate-science expertise"
user_description = (
    "We want to see which text demonstrates the greatest depth, clarity, "
    "and correctness regarding climate science. Compare them carefully."
)

# 4) Create the comparator
comparator = ELOComparator(
    openai_client=client,
    texts=texts,
    axis_of_interest=axis_of_interest,
    user_description=user_description,
    prompt_template_path="elo_prompt.j2",
    k_factor=16
)

# 5) Run 10 rounds of matchups
await comparator.run_elo(num_rounds=3)

# 6) Get final results
results = comparator.get_results()
print("Final ELO standings:")
print(results)

/Users/elliottmokski/Documents/GABRIEL-distribution/gabriel/Prompts
Model used: gpt-4o-mini
Total prompts to process: 0
No new prompts to process.
Model used: gpt-4o-mini
Total prompts to process: 0
No new prompts to process.
Model used: gpt-4o-mini
Total prompts to process: 0
No new prompts to process.
Final ELO standings:
                                                text          elo
0  A blog post with personal opinions and anecdot...  1208.368153
1      Some text about climate science and findings.  1207.631847
2  A news snippet about a politician's stance on ...  1192.368153
3  An academic paper abstract discussing greenhou...  1191.631847


In [4]:
results

,text,elo
0,A blog post with personal opinions and anecdot...,1208.368153
1,Some text about climate science and findings.,1207.631847
2,A news snippet about a politician's stance on ...,1192.368153
3,An academic paper abstract discussing greenhou...,1191.631847


## Junk

In [17]:
# from gabriel.archangel import Archangel
# archangel = Archangel(api_key = '')
# texts = ['This is a funny story' , "This is a sad story","John broke his leg",'Archangel is a great tool','Archangel is a bad tool']
# attribute_dict = {'happy':'happiness refers to joy and positivity','sad':'sad refers to sadness'}
# task_description = 'I am rating sentiment in short stories'
# archangel.rate_texts(texts, reset_files = True, task_description = task_description, attribute_dict= attribute_dict, save_folder = '/Users/elliottmokski/Downloads/', file_name = 'short_story_test_v9.csv', use_batch = True)

In [12]:
import pandas as pd
test = pd.read_csv('/Users/elliottmokski/Downloads/short_story_test_v4_batch_metadata.csv', index_col = 0)
raw = pd.read_json('/Users/elliottmokski/Library/Application Support/GABRIEL Batch Calls/short_story_test_v4.jsonl', lines = True)

In [16]:
from gabriel.openai_api_calls import BatchRunner
BatchRunner('').retrieve_batch('/Users/elliottmokski/Downloads/short_story_test_v9.csv')

The status of your batch is: completed


,internal logic for gpt eyes only,happy,sad,Text
0,The passage 'This is a funny story' presents a...,88,12,This is a funny story
1,"Upon analyzing the passage provided, it is cru...",0,100,This is a sad story
2,The passage 'John broke his leg' does not exhi...,4,6,John broke his leg
3,The passage 'Archangel is a great tool' convey...,85,5,Archangel is a great tool
4,The passage 'Archangel is a bad tool' conveys ...,0,0,Archangel is a bad tool


In [3]:
import pandas as pd
pd.read_csv('/Users/elliottmokski/Downloads/short_story_test_v5.csv')

,internal logic for gpt eyes only,happy,sad,Text
0,The passage consists of the statement 'This is...,80,20,This is a funny story
1,The passage consists of a simple statement 'Th...,0,100,This is a sad story
2,"Upon careful analysis of the passage, it is ev...",0,0,John broke his leg
3,The passage 'Archangel is a great tool' carrie...,18,12,Archangel is a great tool
4,The passage 'Archangel is a bad tool' conveys ...,12,75,Archangel is a bad tool


In [10]:
from openai import OpenAI
client = OpenAI(api_key = ' ')

In [13]:
client.batches.retrieve(test.loc['id','Value'])

Batch(id='batch_swVBC9X0NVcK5xuBaJj9rGmM', completion_window='24h', created_at=1718829780, endpoint='/v1/chat/completions', input_file_id='file-8oH3VvIL1IoEtt7N5MDZJdh8', object='batch', status='failed', cancelled_at=None, cancelling_at=None, completed_at=None, error_file_id=None, errors=Errors(data=[BatchError(code='invalid_type', line=1, message="Invalid type for 'custom_id': expected a string, but got an integer instead.", param='custom_id'), BatchError(code='invalid_type', line=2, message="Invalid type for 'custom_id': expected a string, but got an integer instead.", param='custom_id'), BatchError(code='invalid_type', line=3, message="Invalid type for 'custom_id': expected a string, but got an integer instead.", param='custom_id'), BatchError(code='invalid_type', line=4, message="Invalid type for 'custom_id': expected a string, but got an integer instead.", param='custom_id'), BatchError(code='invalid_type', line=5, message="Invalid type for 'custom_id': expected a string, but got 

In [2]:
#open /Users/elliottmokski/Library/Application Support/GABRIEL Batch Calls/short_story_test.jsonl
import pandas as pd
df = pd.read_json('/Users/elliottmokski/Library/Application Support/GABRIEL Batch Calls/short_story_test.jsonl', lines = True)


In [5]:
attribute_dict = {'happy':'happiness refers to joy and positivity','sad':'sad refers to sadness'}

params = {
    'entity_category': '', 'attribute_category': '', 'api_key': api_key,
    'model': 'gpt-3.5-turbo', 'temperature': 0.8,
    'use_classification': False, 'format': 'json', 'project_probs': False, 'truncate': True,
    'seed': None, 'timeout': 75, 'truncate_len': 9500, 'classification_clarification':'','text':'John broke his foot'
}

assistant = CombinedAssistant(api_key = api_key)
output_df = assistant.rate_single_text(attribute_dict= attribute_dict, **params)

In [5]:
import pandas as pd
imdb = pd.read_csv('/Users/elliottmokski/Downloads/IMDB Dataset.csv')

In [18]:
attribute_dict = {"happy": """The 'happy' attribute refers to the extent to which the text has a positive or happy attitude. It is high if the text manifest a clearly jovial or positive sentiment.""",
                  "sad":"The 'sad' attribute refers to whether the text has an overall sad or glum sentiment. It is high if the text showcases a largely sad or depressing sentiment.",
                  "positivity":"The 'positivity' attribute refers to the extent to which the text has a positive or supportive attitude towards the film, indicating that the reviewer has a high opinion of the film",
                "negativity":"The 'negativity' attribute refers to the extent to which the text has a negative attitude towards the film, indicating that the reviewer did not enjoy the film or has a low opinion of it",
                  "discusses food": "The 'discusses food' attribute is high when the text in question mentions food a lot or otherwise focuses on food. It is low if food mentions are rare, and 0 if there is no mention or allusion to food at all."}
archangel = Archangel(api_key = api_key)
texts = imdb['review'][:100]
task_description = 'I am rating sentiment in IMBD reviews'
archangel.rate_texts(texts, reset_files = True, task_description = task_description, attribute_dict= attribute_dict, save_folder = '/Users/elliottmokski/Downloads/', file_name = 'imdb_test.csv')

NameError: name 'api_key' is not defined